# Extracting Metadata for Better Document Indexing and Understanding
#### In many cases, especially with long documents, a chunk of text may lack the context necessary to disambiguate the chunk from other similar chunks of text. One method of addressing this is manually labelling each chunk in our dataset or knowledge base. However, this can be labour intensive and time consuming for a large number or continually updated set of documents.
#### To combat this, we use LLMs to extract certain contextual information relevant to the document to better help the retrieval and language models disambiguate similar-looking passages.

In [10]:
from llama_index import ListIndex, LLMPredictor
from langchain import OpenAI
from llama_index import download_loader, VectorStoreIndex, ServiceContext
from llama_index.schema import MetadataMode

In [3]:
llm_predictor = LLMPredictor(
    llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=512)
)

We create a node parser that extracts the document title and hypothetical question embeddings relevant to the document chunk.

We also show how to instantiate the `SummaryExtractor` and `KeywordExtractor`, as well as how to create your own custom extractor 
based on the `MetadataFeatureExtractor` base class

In [29]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    MetadataFeatureExtractor,
)
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)


class CustomExtractor(MetadataFeatureExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": node.metadata["document_title"]
                + "\n"
                + node.metadata["excerpt_keywords"]
            }
            for node in nodes
        ]
        return metadata_list


metadata_extractor = MetadataExtractor(
    extractors=[
        TitleExtractor(nodes=5),
        QuestionsAnsweredExtractor(questions=3),
        # SummaryExtractor(summaries=["prev", "self"]),
        # KeywordExtractor(keywords=10),
        # CustomExtractor()
    ],
)

node_parser = SimpleNodeParser(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

In [7]:
from llama_index import SimpleDirectoryReader, DocumentSummaryIndex

We first load the 10k annual SEC report for Uber and Lyft for the years 2019 and 2020 respectively.

In [11]:
# Note the uninformative document file name, which may be a common scenario in a production setting
uber_docs = SimpleDirectoryReader(input_files=["data/10k-132.pdf"]).load_data()
uber_front_pages = uber_docs[0:3]
uber_content = uber_docs[63:69]
uber_docs = uber_front_pages + uber_content

In [30]:
uber_nodes = node_parser.get_nodes_from_documents(uber_docs)

In [31]:
uber_nodes[1].metadata

{'page_label': '2',
 'file_name': '10k-132.pdf',
 'document_title': 'Uber Technologies, Inc. 2019 Annual Report: Revolutionizing Mobility and Logistics Across 69 Countries and 111 Million MAPCs with $65 Billion in Gross Bookings',
 'questions_this_excerpt_can_answer': '\n\n1. How many countries does Uber Technologies, Inc. operate in?\n2. What is the total number of MAPCs served by Uber Technologies, Inc.?\n3. How much gross bookings did Uber Technologies, Inc. generate in 2019?'}

In [12]:
# Note the uninformative document file name, which may be a common scenario in a production setting
lyft_docs = SimpleDirectoryReader(input_files=["data/10k-vFinal.pdf"]).load_data()
lyft_front_pages = lyft_docs[0:3]
lyft_content = lyft_docs[68:73]
lyft_docs = lyft_front_pages + lyft_content

In [13]:
lyft_nodes = node_parser.get_nodes_from_documents(lyft_docs)

In [14]:
lyft_nodes[2].metadata

{'page_label': '2',
 'file_name': '10k-vFinal.pdf',
 'document_title': "Lyft, Inc. Filing and Attestation of Management's Assessment Report, Filer Status and Stock Information, and Proxy Statement Incorporation in Annual Report on Form 10-K with Historical Consolidated Statements of Operations Data",
 'questions_this_excerpt_can_answer': "\n1. What is the status of the registrant as a filer?\n2. Has the registrant filed a report on and attestation to its management's assessment of the effectiveness of its internal control over financial reporting?\n3. What is the aggregate market value of the Registrant's common stock held by non-affiliates of the Registrant on June 30, 2020?"}

Since we are asking fairly sophisticated questions, we utilize a subquestion query engine for all QnA pipelines below, and prompt it to pay more attention to the relevance of the retrieved sources. 

In [16]:
from llama_index.question_gen.llm_generators import LLMQuestionGenerator
from llama_index.question_gen.prompts import DEFAULT_SUB_QUESTION_PROMPT_TMPL

service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor, node_parser=node_parser
)
question_gen = LLMQuestionGenerator.from_defaults(
    service_context=service_context,
    prompt_template_str="""
        Follow the example, but instead of giving a question, always prefix the question 
        with: 'By first identifying and quoting the most relevant sources, '. 
        """
    + DEFAULT_SUB_QUESTION_PROMPT_TMPL,
)

## Querying an Index With No Extra Metadata

In [17]:
from copy import deepcopy

nodes_no_metadata = deepcopy(uber_nodes) + deepcopy(lyft_nodes)
for node in nodes_no_metadata:
    node.metadata = {
        k: node.metadata[k] for k in node.metadata if k in ["page_label", "file_name"]
    }
print("LLM sees:\n", (nodes_no_metadata)[9].get_content(metadata_mode=MetadataMode.LLM))

LLM sees:
 [Excerpt from document]
page_label: 65
file_name: 10k-132.pdf
            Excerpt:
-----
See the section titled “Reconciliations of Non-GAAP Financial Measures” for our definition and a 
reconciliation of net income (loss) attributable to  Uber Technologies, Inc. to Adjusted EBITDA. 
            
  Year Ended December 31,   2017 to 2018   2018 to 2019   
(In millions, exce pt percenta ges)  2017   2018   2019   % Chan ge  % Chan ge  
Adjusted EBITDA ................................  $ (2,642) $ (1,847) $ (2,725)  30%  (48)%
-----


In [18]:
from llama_index import VectorStoreIndex
from llama_index.vector_stores import FaissVectorStore
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.tools import QueryEngineTool, ToolMetadata

In [19]:
index_no_metadata = VectorStoreIndex(nodes=nodes_no_metadata)
engine_no_metadata = index_no_metadata.as_query_engine(similarity_top_k=10)

In [20]:
final_engine_no_metadata = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine_no_metadata,
            metadata=ToolMetadata(
                name="sec_filing_documents",
                description="financial information on companies",
            ),
        )
    ],
    question_gen=question_gen,
    use_async=False,
)

In [21]:
response_no_metadata = final_engine_no_metadata.query(
    """
    What was the cost due to research and development v.s. sales and marketing for uber and lyft in 2019 in millions of USD?
    Give your answer as a JSON.
    """
)
response_no_metadata.response
# Correct answer:
# {"Uber": {"Research and Development": 4836, "Sales and Marketing": 4626},
#  "Lyft": {"Research and Development": 1505.6, "Sales and Marketing": 814 }}

Generated 4 sub questions.
[sec_filing_documents] Q: By first identifying and quoting the most relevant sources, what was the cost due to research and development for Uber in 2019 in millions of USD?
[sec_filing_documents] A: 

According to the excerpt from page 70 of the document 10k-vFinal.pdf, research and development expenses for Uber in 2019 were $909.1 million in thousands of USD, before income taxes and loss from equity method investment. Therefore, the cost due to research and development for Uber in 2019 in millions of USD was $909.1 million.
[sec_filing_documents] Q: By first identifying and quoting the most relevant sources, what was the cost due to sales and marketing for Uber in 2019 in millions of USD?
[sec_filing_documents] A: 

According to the document "10k-vFinal.pdf" on page 69, the cost due to sales and marketing for Uber in 2019 was $397.8 million in thousands of USD, or $397,800,000 in millions of USD. This cost was part of the overall increase in Adjusted EBITDA 

'Answer: \n{\n    "Uber": {\n        "Research and Development": 909.1,\n        "Sales and Marketing": 397.8\n    },\n    "Lyft": {\n        "Research and Development": 232.1,\n        "Sales and Marketing": 814.1\n    }\n}'

#### **RESULT**: As we can see, the QnA agent does not seem to know where to look for the right documents. As a result it gets only 1/4 of the subquestions right.

## Querying an Index With Extracted Metadata

In [22]:
print(
    "LLM sees:\n",
    (uber_nodes + lyft_nodes)[9].get_content(metadata_mode=MetadataMode.LLM),
)

LLM sees:
 [Excerpt from document]
page_label: 65
file_name: 10k-132.pdf
document_title: Uber Technologies, Inc. 2019 Annual Report: Revolutionizing Mobility and Logistics with Technology and Infrastructure.
            Excerpt:
-----
See the section titled “Reconciliations of Non-GAAP Financial Measures” for our definition and a 
reconciliation of net income (loss) attributable to  Uber Technologies, Inc. to Adjusted EBITDA. 
            
  Year Ended December 31,   2017 to 2018   2018 to 2019   
(In millions, exce pt percenta ges)  2017   2018   2019   % Chan ge  % Chan ge  
Adjusted EBITDA ................................  $ (2,642) $ (1,847) $ (2,725)  30%  (48)%
-----


In [23]:
index = VectorStoreIndex(nodes=uber_nodes + lyft_nodes)
engine = index.as_query_engine(similarity_top_k=10)

In [24]:
final_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name="sec_filing_documents",
                description="financial information on companies.",
            ),
        )
    ],
    question_gen=question_gen,
    use_async=False,
)

In [25]:
response = final_engine.query(
    """
    What was the cost due to research and development v.s. sales and marketing for uber and lyft in 2019 in millions of USD?
    Give your answer as a JSON.
    """
)
response.response
# Correct answer:
# {"Uber": {"Research and Development": 4836, "Sales and Marketing": 4626},
#  "Lyft": {"Research and Development": 1505.6, "Sales and Marketing": 814 }}

Generated 4 sub questions.
[sec_filing_documents] Q: By first identifying and quoting the most relevant sources, what was the cost due to research and development for Uber in 2019 in millions of USD?
[sec_filing_documents] A: 

According to the excerpt from page 69 of the document "Uber Technologies, Inc. 2019 Annual Report: Revolutionizing Mobility and Logistics with Technology and Infrastructure.", the cost due to research and development for Uber in 2019 was $4.836 billion in millions of USD. This is stated in the following excerpt: "Research and development .....................................................................  $1,201  $1,505  $4,836". This cost is exclusive of depreciation and amortization, and includes fees, hosting and co-located data center expenses, and other cost of revenue. It is driven by investments in the business, including excess Driver incentives, and the Freight offering and New Mobility products.
[sec_filing_documents] Q: By first identifying and quot

'Answer: \n{\n    "Uber": {\n        "Research and Development": 4836,\n        "Sales and Marketing": 4626\n    },\n    "Lyft": {\n        "Research and Development": 1505,\n        "Sales and Marketing": 814.122\n    }\n}'

#### **RESULT**: As we can see, the LLM answers the questions correctly.

### Challenges Identified in the Problem Domain

In this example, we observed that the search quality as provided by vector embeddings was rather poor. This was likely due to highly dense financial documents that were likely not representative of the training set for the model.

In order to improve the search quality, other methods of neural search that employ more keyword-based approaches may help, such as ColBERTv2/PLAID. In particular, this would help in matching on particular keywords to identify high-relevance chunks.

Other valid steps may include utilizing models that are fine-tuned on financial datasets such as Bloomberg GPT.

Finally, we can help to further enrich the metadata by providing more contextual information regarding the surrounding context that the chunk is located in.

### Improvements to this Example
Generally, this example can be improved further with more rigorous evaluation of both the metadata extraction accuracy, and the accuracy and recall of the QnA pipeline. Further, incorporating a larger set of documents as well as the full length documents, which may provide more confounding passages that are difficult to disambiguate, could further stresss test the system we have built and suggest further improvements. 